<a href="https://colab.research.google.com/github/donghyuun/ProjectAW/blob/main/tft_v4_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
!pip install pytorch-forecasting

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.9/181.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 54.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import torch
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import RMSE
from pytorch_lightning import Trainer
import pytorch_lightning as pl
from torch.optim import Adam
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.callbacks import ModelCheckpoint  # ModelCheckpoint 임포트
from pytorch_lightning.callbacks.early_stopping import EarlyStopping  # EarlyStopping 콜백도 임포트

# Google Drive에서 데이터 로드
train_data_path = "train_data.csv"
test_data_path = "test_data.csv"

# 데이터 로드
df_train = pd.read_csv(train_data_path)
df_test = pd.read_csv(test_data_path)
df_final = pd.read_csv("final_final_preprocessed_with_updated_time_format.csv")

# 스케일링
scaler = MinMaxScaler()
df_train['feed_pressure'] = scaler.fit_transform(df_train[['feed_pressure']])
df_test['feed_pressure'] = scaler.transform(df_test[['feed_pressure']])
df_final['feed_pressure'] = scaler.fit_transform(df_final[['feed_pressure']])

# 시간 인덱스 추가
df_train['time_idx'] = range(len(df_train))
df_test['time_idx'] = range(len(df_train), len(df_train) + len(df_test))
df_final['time_idx'] = range(len(df_final))

# 학습 및 테스트 데이터 병합
df_total = pd.concat([df_train, df_test], ignore_index=True)

# max_encoder_length와 max_prediction_length 설정
total_data_length = len(df_train)
# max_encoder_length = max(10, total_data_length // 10)  #12시간
max_encoder_length = 1000  #12시간
max_prediction_length = 1  #2시간

print(f"Max encoder length: {max_encoder_length} entries")
print(f"Max prediction length: {max_prediction_length} entries")

# 그룹 ID 설정
df_total["group"] = "series"  # 단일 시계열일 경우 고유 그룹 지정

# TimeSeriesDataSet 생
training = TimeSeriesDataSet(
    df_total[lambda x: x.time_idx < len(df_train)],  # 학습 데이터만 사용
    time_idx="time_idx",
    target="feed_pressure",  # 타겟 변수
    group_ids=["group"],  # 그룹화
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=[],
    time_varying_known_reals=["time_idx"],  # 시간 관련 변수
    time_varying_unknown_reals=["feed_pressure"],  # 예측 대상 변수
    target_normalizer=GroupNormalizer(transformation="relu")  # 정규화
)

# 데이터 로더 재생성
train_dataloader = training.to_dataloader(train=True, batch_size=32, num_workers=0)
val_dataloader = training.to_dataloader(train=False, batch_size=32, num_workers=0)

from torchmetrics import MeanAbsoluteError

# Define the loss function
loss = MeanAbsoluteError()

# Temporal Fusion Transformer 모델 생성
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.001,  # 학습률
    hidden_size=16,  # 모델의 잠재 표현 크기
    attention_head_size=1,  # 어텐션 헤드 개수 (패턴 포착)
    dropout=0.3,  # 과적합 방지
    hidden_continuous_size=16,  # 연속형 변수의 임베딩 크기
    loss=loss,  # 손실 함수 (loss 변수 사용)
    log_interval=10,  # 학습 로그 출력 빈도
    reduce_on_plateau_patience=2,  # 손실이 개선되지 않을 시 학습률 줄이는 시점
    optimizer="adam"
)

class TFTWrapper(pl.LightningModule):
    def __init__(self, tft_model):
        super().__init__()
        self.tft_model = tft_model
        self.train()  # 명시적으로 학습 모드로 설정

    def forward(self, x):
        # 모든 입력 데이터를 모델의 디바이스로 이동
        x = {key: value.to(self.device) if isinstance(value, torch.Tensor) else value for key, value in x.items()}
        return self.tft_model(x)

    def training_step(self, batch, batch_idx):
        # 모든 데이터를 모델의 디바이스로 이동
        self.train()  # 학습 스텝에서 다시 학습 모드로 전환
        x, y = batch
        x = {key: value.to(self.device) if isinstance(value, torch.Tensor) else value for key, value in x.items()}

        # y가 튜플인 경우, 첫 번째 요소만 가져오기
        if isinstance(y, tuple):
            y = y[0]
        y = y.to(self.device)

        y_hat = self(x)["prediction"]
        loss = self.tft_model.loss(y_hat, y)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        # 모든 데이터를 모델의 디바이스로 이동
        x, y = batch
        x = {key: value.to(self.device) if isinstance(value, torch.Tensor) else value for key, value in x.items()}

        # y가 튜플인 경우, 첫 번째 요소만 가져오기
        if isinstance(y, tuple):
            y = y[0]
        y = y.to(self.device)

        y_hat = self(x)["prediction"]
        loss = self.tft_model.loss(y_hat, y)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.03)
        lr_scheduler = {
            'scheduler': torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=5),
            'monitor': 'val_loss'
        }
        return [optimizer], [lr_scheduler]

# TFT 모델을 TFTWrapper로 감싸기
# tft.to('cuda')
# wrapped_tft = TFTWrapper(tft).to('cuda')
tft.to('cuda')
wrapped_tft = TFTWrapper(tft).to('cuda')

# Early Stopping Callback 추가
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=1e-4,
    patience=10,
    verbose=True,
    mode="min"
)


# 학습 실행 시 체크포인트 콜백 포함
trainer = Trainer(
    max_epochs=5,
    accelerator="gpu",
    devices=1,
    gradient_clip_val=0.3,
    precision=32,
    callbacks=early_stop_callback,  # 콜백 추가
)

# 새 학습 시작
trainer.fit(
    model=wrapped_tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader
)

# 모델 및 데이터셋 저장
MODEL_PATH = "model_checkpoint.pth"
SCALER_PATH = "scaler.pkl"
DATASET_PATH = "timeseries_dataset.pkl"

# 모델 상태 및 데이터셋 저장
print("[INFO] Saving model and dataset...")
torch.save({
    'model_state_dict': tft.state_dict(),  # 모델 상태
    'timeseries_dataset': training          # TimeSeriesDataSet 객체
}, MODEL_PATH)
print(f"[INFO] Model and dataset saved to {MODEL_PATH}")

# 스케일러 저장
print("[INFO] Saving scaler...")
import joblib
joblib.dump(scaler, SCALER_PATH)
print(f"[INFO] Scaler saved to {SCALER_PATH}")

# TimeSeriesDataSet 별도로 저장
print("[INFO] Saving TimeSeriesDataSet...")
with open(DATASET_PATH, "wb") as f:
    joblib.dump(training, f)
print(f"[INFO] TimeSeriesDataSet saved to {DATASET_PATH}")


Max encoder length: 1000 entries
Max prediction length: 1 entries


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/data.py:78: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 32. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

...

INFO:pytorch_lightning.callbacks.early_stopping:Metric val_loss improved by 0.001 >= min_delta = 0.0001. New best score: 0.059


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


[INFO] Saving model and dataset...
[INFO] Model and dataset saved to model_checkpoint.pth
[INFO] Saving scaler...
[INFO] Scaler saved to scaler.pkl
[INFO] Saving TimeSeriesDataSet...
[INFO] TimeSeriesDataSet saved to timeseries_dataset.pkl


In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# 'group' 컬럼 확인 및 추가
if "group" not in df_test.columns or df_test["group"].isnull().any():
    df_test["group"] = "series"  # 기본 그룹 값 설정

# Rolling Prediction 실행 전에 데이터 유효성 확인
if len(df_test) < max_encoder_length:
    raise ValueError(
        f"Test dataset length ({len(df_test)}) is shorter than max_encoder_length ({max_encoder_length})."
    )

# 'group' 컬럼 확인 및 추가
if "group" not in df_final.columns or df_final["group"].isnull().any():
    df_final["group"] = "series"  # 기본 그룹 값 설정

# Rolling Prediction 실행 전에 데이터 유효성 확인
if len(df_final) < max_encoder_length:
    raise ValueError(
        f"Test dataset length ({len(df_final)}) is shorter than max_encoder_length ({max_encoder_length})."
    )

def rolling_predict_until_condition(
    model, train_dataset, max_encoder_length, scaler, condition_value=17, tolerance=0.1, device="cuda"
):
    """
    Rolling 방식으로 train의 첫 max_encoder_length 데이터를 바탕으로 조건 만족 시까지 예측합니다.

    Args:
        model: PyTorch 모델.
        train_dataset: 학습에 사용된 데이터셋 (DataFrame 형태).
        max_encoder_length: 롤링 윈도우의 길이.
        scaler: MinMaxScaler (스케일링 복원을 위해 필요).
        condition_value: 예측이 중단되는 조건 값 (예: 17).
        tolerance: 조건 값에 대한 허용 오차.
        device: 사용할 디바이스 (default: "cuda").

    Returns:
        predictions (list): 예측된 값의 리스트.
    """
    model = model.to(device)
    model.eval()

    # Initialize with the first encoder window from the training data
    input_data = train_dataset.iloc[:max_encoder_length].copy()
    print("max_encoder_length: ", max_encoder_length)
    predictions = []

    # Add group column if missing
    if "group" not in input_data.columns:
        input_data["group"] = "series"

    step = 0
    while True:
        try:
            # Create TimeSeriesDataSet for rolling prediction
            rolling_dataset = TimeSeriesDataSet(
                data=input_data,
                time_idx="time_idx",
                target="feed_pressure",
                group_ids=["group"],
                max_encoder_length=max_encoder_length,
                max_prediction_length=1,
                static_categoricals=[],
                time_varying_known_reals=["time_idx"],
                time_varying_unknown_reals=["feed_pressure"],
                target_normalizer=GroupNormalizer(transformation="relu"),
                min_encoder_length=1,
                allow_missing_timesteps=True,
            )

            rolling_dataloader = rolling_dataset.to_dataloader(train=False, batch_size=1)

            # Predict the next value
            with torch.no_grad():
                input_batch = next(iter(rolling_dataloader))
                input_batch = {key: value.to(device) for key, value in input_batch[0].items()}
                prediction = model(input_batch)["prediction"].cpu().numpy().squeeze()

            # Restore the scaled prediction to original scale
            prediction_actual = scaler.inverse_transform([[prediction]]).flatten()[0]
            predictions.append(prediction_actual)

            print(f"Step {step}: Predicted Value = {prediction_actual}")

            # Check if the condition is met
            if abs(prediction_actual - condition_value) <= tolerance:
                print(f"Condition met: Predicted value is within tolerance of {condition_value}. Stopping prediction.")
                break

            # Update rolling input data with new predicted value
            new_time_idx = input_data["time_idx"].iloc[-1] + 1
            new_row = {
                "time_idx": new_time_idx,
                "feed_pressure": float(prediction),
                "group": "series",
            }
            input_data = pd.concat([input_data.iloc[1:], pd.DataFrame([new_row])], ignore_index=True)

            step += 1

        except Exception as e:
            print(f"Error at step {step}: {e}")
            break

    return predictions

# 마지막 데이터의 'time' 컬럼 값 출력
from datetime import datetime

# 시간 포맷 통일 함수
def format_time(value, current_format="%Y %m %d %H:%M:%S", target_format="%Y년 %m월 %d일 %H시 %M분"):
    """
    시간 문자열을 원하는 형식으로 변환.

    Args:
        value (str): 원본 시간 문자열.
        current_format (str): 원본 시간의 형식.
        target_format (str): 변환할 시간 형식.

    Returns:
        str: 변환된 시간 문자열.
    """
    time_object = datetime.strptime(value, current_format)  # 원본 시간 형식으로 변환
    return time_object.strftime(target_format)  # 타겟 형식으로 출력

print(df_train.columns)
print(df_test.columns)
print(df_final.columns)

# 'time' 컬럼의 마지막 값을 지정된 포맷으로 출력
last_time_value = df_final['time'].iloc[892]
formatted_last_time = format_time(last_time_value, current_format="%Y %m %d %H:%M:%S", target_format="%Y년 %m월 %d일 %H시 %M분")

print(f"예측용 데이터의 마지막 시간: {formatted_last_time}")

# Rolling Prediction 함수 실행 후 조건 만족 시 시간 계산 및 출력
from datetime import datetime, timedelta

def calculate_future_time(last_time_value, steps, step_duration_minutes=1):
    # 시간 문자열을 datetime 객체로 변환
    last_time_dt = datetime.strptime(last_time_value, "%Y %m %d %H:%M:%S")  # 포맷 수정

    # 총 시간 간격 계산
    total_duration = timedelta(minutes=steps * step_duration_minutes)

    # 미래 시간 계산
    future_time = last_time_dt + total_duration

    # 년, 월, 일, 시, 분 형식으로 반환
    return future_time.strftime("%Y년 %m월 %d일 %H시 %M분")


# Rolling Prediction 실행
rolling_predictions = rolling_predict_until_condition(
    model=wrapped_tft,
    train_dataset=df_final,
    max_encoder_length=max_encoder_length,
    scaler=scaler,
    condition_value=10,
    tolerance=0.1,
    device="cuda",
)
condition_value=10

# 총 steps 계산
total_steps = len(rolling_predictions)

# 임계점 도달 시 시간 계산
future_time = calculate_future_time(last_time_value, total_steps)

# 결과 출력
print(f"Total Steps: {total_steps}")
print(f"RO 유입 압력이 {condition_value} bar에 도달할 예정 날짜는 {future_time}입니다.")


Index(['time', 'feed_pressure', 'time_idx'], dtype='object')
Index(['time', 'feed_pressure', 'time_idx', 'group'], dtype='object')
Index(['time', 'feed_pressure', 'time_idx', 'group'], dtype='object')
예측용 데이터의 마지막 시간: 2024년 10월 08일 07시 32분
max_encoder_length:  1000
Step 0: Predicted Value = 9.588438481461257
Step 1: Predicted Value = 9.589090145096183
Step 2: Predicted Value = 9.589723205683752
Step 3: Predicted Value = 9.590321733668446
Step 4: Predicted Value = 9.590887956960126
Step 5: Predicted Value = 9.591418310903013
Step 6: Predicted Value = 9.59187157916464
Step 7: Predicted Value = 9.592217127984389
Step 8: Predicted Value = 9.592471666686237
Step 9: Predicted Value = 9.592645889237524
Step 10: Predicted Value = 9.592759846827015
Step 11: Predicted Value = 9.593248650250956
Step 12: Predicted Value = 9.593928942527622
Step 13: Predicted Value = 9.594665489528328
Step 14: Predicted Value = 9.595361265778541
Step 15: Predicted Value = 9.595941079320387
Step 16: Predicted Value 

In [ ]:
!pip show pytorch-lightning


Name: pytorch-lightning
Version: 2.4.0
Summary: PyTorch Lightning is the lightweight PyTorch wrapper for ML researchers. Scale your models. Write less boilerplate.
Home-page: https://github.com/Lightning-AI/lightning
Author: Lightning AI et al.
Author-email: pytorch@lightning.ai
License: Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: fsspec, lightning-utilities, packaging, PyYAML, torch, torchmetrics, tqdm, typing-extensions
Required-by: lightning


</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
